<div class="alert alert-block alert-info">
<b>Importing Necessary Libraries:</b> We will import necessary libraries for analysis. One of them being Skopt which will be used for parameter tuning of the Seq2Seq model.
</div>

In [20]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from random import shuffle
import os
import re
from keras.models import Sequential,load_model
from sklearn.model_selection import train_test_split
from keras.layers import Bidirectional,Dense, CuDNNGRU, CuDNNLSTM, RepeatVector, TimeDistributed, BatchNormalization, Embedding
from keras.optimizers import *
from keras.callbacks import *
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

import skopt
from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
from skopt.plots import plot_objective
from skopt.utils import use_named_args

<div class="alert alert-block alert-info">
<b>Loading the data for analysis :</b> We will load the data for analysis. We will extract the Question-Answer pair from the data. 
</div>

In [2]:
df_professional = pd.read_csv('qna_chitchat_professional.tsv',sep='\t')

In [3]:
df_professional.head(4)

,Question,Answer,Source,Metadata
0,Are we the same age?,Age doesn't really apply to me.,qna_chitchat_professional,editorial:chitchat
1,Are you a baby?,Age doesn't really apply to me.,qna_chitchat_professional,editorial:chitchat
2,Are you a grown up?,Age doesn't really apply to me.,qna_chitchat_professional,editorial:chitchat
3,Are you a grownup?,Age doesn't really apply to me.,qna_chitchat_professional,editorial:chitchat


<div class="alert alert-block alert-info">
<b>Data Cleaning(Text Cleaning):</b> We will perform text preprocessing on the data loaded. This will involve removal of punctuations, numerics and converting all the words to lowercase.
</div>

In [4]:
clean_text                  = []
for i in range(len(df_professional)):
    ques_text               = df_professional.loc[i]['Question']
    ans_text                = df_professional.loc[i]['Answer']
    
    ques_text_split         = ques_text.split()
    ans_text_split          = ans_text.split()
    
    ques_text__punc_removed = ' '.join([re.sub('[^A-Za-z]+', '', ques_text_split[i]) for i in range(len(ques_text_split))])
    ans_text_punc_removed   = ' '.join([re.sub('[^A-Za-z]+', '', ans_text_split[i]) for i in range(len(ans_text_split))])
    
    
    ques_text_nums_removed  = ' '.join([word.lower() for word in ques_text__punc_removed.split() if word.isalpha()])
    ans_text_nums_removed   = ' '.join([word.lower() for word in ans_text_punc_removed.split() if word.isalpha()])
    
    clean_text.append([ques_text_nums_removed,ans_text_nums_removed])
clean_text                  = np.array(clean_text)
print('Punctuations removed and all the sentences converted to smaller case.')

Punctuations removed and all the sentences converted to smaller case.


<div class="alert alert-block alert-info">
<b>Tokenization and Vocabulary creation:</b> We will tokenize the text and also create separate vocabularies for Questions and Answers to incorporate Seq2seq architecture. We will then find out T_x (maximum size of Question or Input statement) to the Encoder model and T_y (maximum size of Answer or Output statement) from the Decoder Model
</div>

In [5]:
ques_tokenizer       = Tokenizer()
ques_tokenizer.fit_on_texts(clean_text[:,0])
question_vocab       = len(ques_tokenizer.word_index) + 1
question_length      = max([len(clean_text[:,0][i].split()) for i in range(len(clean_text[:,0]))])
print('The number of distinct words in the Questions is ',question_vocab)
print('The maximum length of sentence in Question is    ',question_length)

The number of distinct words in the Questions is  2028
The maximum length of sentence in Question is     15


In [6]:
ans_tokenizer        = Tokenizer()
ans_tokenizer.fit_on_texts(clean_text[:,1])
ans_vocab            = len(ans_tokenizer.word_index) + 1
ans_length           = max([len(clean_text[:,1][i].split()) for i in range(len(clean_text[:,1]))])
print('The number of distinct words in the Answers is ',ans_vocab)
print('The maximum length of sentence in Answers is   ',ans_length)

The number of distinct words in the Answers is  196
The maximum length of sentence in Answers is    12


<div class="alert alert-block alert-info">
<b>Encoding sequences:</b> We will now encode the sequences of questions and answers and split them to train and test data
</div>

In [7]:
def encode_sequences(tokenizer, length, lines):
    X = tokenizer.texts_to_sequences(lines)
    X = pad_sequences(X, maxlen=length, padding='post')
    return X
def encode_output(sequences, vocab_size):
    y_list = []
    for sequence in sequences:
        encoded = to_categorical(sequence, num_classes=vocab_size)
        y_list.append(encoded)
    y = np.array(y_list)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y

In [8]:
X = encode_sequences(ques_tokenizer, question_length, clean_text[:, 0])
Y = encode_sequences(ans_tokenizer, ans_length, clean_text[:, 1])
Y = encode_output(Y, ans_vocab)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

<div class="alert alert-block alert-info">
<b>Model Creation:</b> We write a method to create a model with some parameters to be input while we do Hyper parameter tuning
</div>

In [10]:
def create_model(learning_rate,num_rnn_nodes,emb_size):
    model  = Sequential()
    model.add(Embedding(question_vocab, emb_size, input_length=question_length))
    model.add(BatchNormalization())
    model.add(Bidirectional(CuDNNLSTM(num_rnn_nodes)))
    model.add(BatchNormalization())
    model.add(RepeatVector(ans_length))
    model.add(Bidirectional(CuDNNLSTM(num_rnn_nodes, return_sequences=True)))
    model.add(BatchNormalization())
    model.add(TimeDistributed(Dense(ans_vocab, activation='softmax')))
    model.compile(optimizer=adam(lr=learning_rate), loss='categorical_crossentropy', metrics = ['accuracy'])
    return model


<div class="alert alert-block alert-info">
<b>Hyper parameter margin and initialization:</b> We set up initial values of the Hyper parameters to start with and also specify the range within which we want to find the values of the hyper parameters
</div>

In [11]:
learning_rate      = Real(low=1e-6, high=1e-2, prior='log-uniform',name='learning_rate')
num_rnn_nodes      = Integer(low=20, high=800, name='num_rnn_nodes')
emb_size           = Integer(low=10,high=100,name='emb_size')
#batch_size         = Integer(low=4,high=128,name='batch_size')
dimensions         = [learning_rate,num_rnn_nodes,emb_size]
default_parameters = [0.005, 600, 20]
best_accuracy      = 0.0
path_best_model    = 'chatbot_best_model.keras'

<div class="alert alert-block alert-info">
<b>Fitting and Hyper parameter tuning:</b> The below function performs the hyper parameter tuning and by calling the function to create the model prepared above, setting callbacks and then finally updating the accuracy based on current and previous results for accuracy 
</div>

In [12]:
@use_named_args(dimensions=dimensions)
def fitness(learning_rate, num_rnn_nodes,emb_size):
    print('learning rate: {0:.1e}'.format(learning_rate))
    print('RNN Nodes:', num_rnn_nodes)
    print('Embedding size:', emb_size)
    
    model = create_model(learning_rate=learning_rate,
                         num_rnn_nodes=num_rnn_nodes,
                         emb_size=emb_size)

    callback_log = TensorBoard(
        histogram_freq=0,
        batch_size=32,
        write_graph=True,
        write_grads=False,
        write_images=False)
    es         = EarlyStopping(monitor='val_acc', patience=15, verbose=1, mode='auto', baseline=None, 
                          restore_best_weights=True)   
    history = model.fit(X_train,y_train,
                        epochs=30,
                        batch_size=32,
                        validation_data=(X_test,y_test),
                        callbacks=[callback_log]+[es])
    accuracy = max(history.history['val_acc'])
    print("Accuracy: ",accuracy)
    global best_accuracy
    if accuracy > best_accuracy:
        model.save(path_best_model)
        best_accuracy = accuracy
    del model
    K.clear_session()
    return -accuracy

In [13]:
search_result = gp_minimize(func=fitness,
                            dimensions=dimensions,
                            acq_func='EI', # Expected Improvement.
                            n_calls=40,
                            x0=default_parameters)

learning rate: 5.0e-03
RNN Nodes: 600
Embedding size: 20
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 4477 samples, validate on 2206 samples
Epoch 1/30
4477/4477 [==============================] - 10s 2ms/step - loss: 2.3618 - acc: 0.5931 - val_loss: 16.0560 - val_acc: 7.5552e-04
Epoch 2/30
4477/4477 [==============================] - 8s 2ms/step - loss: 1.2547 - acc: 0.7174 - val_loss: 15.9829 - val_acc: 0.0028
Epoch 3/30
4477/4477 [==============================] - 8s 2ms/step - loss: 0.8321 - acc: 0.7859 - val_loss: 16.0300 - val_acc: 0.0013
Epoch 4/30
4477/4477 [==============================] - 8s 2ms/step - loss: 0.6022 - acc: 0.8348 - val_loss: 10.8895 - val_acc: 0.0427
Epoch 5/30
4477/4477 [==============================] - 8s 2ms/step - loss: 0.4607 - acc: 0.8704 - val_loss: 15.9110 - val_acc: 0.0028
Epoch 6/30
4477/4477 [==============================] - 8s 2ms/step - loss: 0.3611 - acc: 0.896

4477/4477 [==============================] - 10s 2ms/step - loss: 0.0426 - acc: 0.9879 - val_loss: 0.7634 - val_acc: 0.87230.0388 - ETA: 1s - loss: 0.041 - ETA: 0s - loss: 0.0416 - a
Epoch 17/30
4477/4477 [==============================] - 10s 2ms/step - loss: 0.0472 - acc: 0.9871 - val_loss: 0.6714 - val_acc: 0.8908 0s - loss: 0.0472 - acc: 0
Epoch 18/30
4477/4477 [==============================] - 10s 2ms/step - loss: 0.0458 - acc: 0.9866 - val_loss: 0.7533 - val_acc: 0.8744
Epoch 19/30
4477/4477 [==============================] - 10s 2ms/step - loss: 0.0508 - acc: 0.9851 - val_loss: 0.8199 - val_acc: 0.8689: 0.0 - ETA: 4s - loss: 0.0426 - ETA: 3s - loss: 0.0442 - acc: 0.9 - ETA: 3s - ETA: 1s - loss: 
Epoch 20/30
4477/4477 [==============================] - 10s 2ms/step - loss: 0.0619 - acc: 0.9839 - val_loss: 0.7601 - val_acc: 0.8793
Epoch 21/30
4477/4477 [==============================] - 10s 2ms/step - loss: 0.0320 - acc: 0.9912 - val_loss: 0.6247 - val_acc: 0.8986
Epoch 22/30
447

4477/4477 [==============================] - 9s 2ms/step - loss: 0.3810 - acc: 0.8864 - val_loss: 6.8712 - val_acc: 0.21680s - loss: 0.3779 - acc: 0
Epoch 11/30
4477/4477 [==============================] - 9s 2ms/step - loss: 0.3404 - acc: 0.8979 - val_loss: 4.8743 - val_acc: 0.4043 - loss: 0.3 - ETA: 4s - loss: 0.3239 - acc: 0.90 - ETA: 4s - loss: 0.3262 - acc: 0. - ETA: 4s - loss:  - ETA: 0s - loss: 0.3362 -
Epoch 12/30
4477/4477 [==============================] - 9s 2ms/step - loss: 0.3079 - acc: 0.9089 - val_loss: 4.4151 - val_acc: 0.4487c
Epoch 13/30
4477/4477 [==============================] - 9s 2ms/step - loss: 0.2907 - acc: 0.9139 - val_loss: 12.3698 - val_acc: 0.0292 
Epoch 14/30
4477/4477 [==============================] - 9s 2ms/step - loss: 0.2927 - acc: 0.9128 - val_loss: 15.8816 - val_acc: 0.0049 - loss: 0.2829 - ac - - ETA:  - ETA: 0s - loss: 0.2902
Epoch 15/30
4477/4477 [==============================] - 9s 2ms/step - loss: 0.2460 - acc: 0.9264 - val_loss: 15.8016 - va

4477/4477 [==============================] - 3s 753us/step - loss: 2.2045 - acc: 0.5946 - val_loss: 2.1883 - val_acc: 0.5965
Epoch 18/30
4477/4477 [==============================] - 3s 753us/step - loss: 2.1341 - acc: 0.6000 - val_loss: 2.1757 - val_acc: 0.5984
Epoch 19/30
4477/4477 [==============================] - 3s 752us/step - loss: 2.0413 - acc: 0.6096 - val_loss: 2.1241 - val_acc: 0.6037
Epoch 20/30
4477/4477 [==============================] - 3s 752us/step - loss: 1.9854 - acc: 0.6169 - val_loss: 2.0751 - val_acc: 0.6092
Epoch 21/30
4477/4477 [==============================] - 3s 752us/step - loss: 1.9303 - acc: 0.6233 - val_loss: 2.0496 - val_acc: 0.6125
Epoch 22/30
4477/4477 [==============================] - 3s 754us/step - loss: 1.8687 - acc: 0.6308 - val_loss: 2.0190 - val_acc: 0.6165s - loss: 1.8914 
Epoch 23/30
4477/4477 [==============================] - 3s 753us/step - loss: 1.8334 - acc: 0.6326 - val_loss: 1.9452 - val_acc: 0.62290 -
Epoch 24/30
4477/4477 [==========

4477/4477 [==============================] - 8s 2ms/step - loss: 2.3159 - acc: 0.5869 - val_loss: 2.3517 - val_acc: 0.5850
Epoch 15/30
4477/4477 [==============================] - 8s 2ms/step - loss: 2.2398 - acc: 0.5944 - val_loss: 2.2735 - val_acc: 0.5931
Epoch 16/30
4477/4477 [==============================] - 8s 2ms/step - loss: 2.1703 - acc: 0.6028 - val_loss: 2.1982 - val_acc: 0.6011
Epoch 17/30
4477/4477 [==============================] - 8s 2ms/step - loss: 2.1314 - acc: 0.6051 - val_loss: 2.2104 - val_acc: 0.5996
Epoch 18/30
4477/4477 [==============================] - 8s 2ms/step - loss: 2.0713 - acc: 0.6126 - val_loss: 2.1327 - val_acc: 0.6057
Epoch 19/30
4477/4477 [==============================] - 8s 2ms/step - loss: 2.0386 - acc: 0.6165 - val_loss: 2.0818 - val_acc: 0.6105
Epoch 20/30
4477/4477 [==============================] - 8s 2ms/step - loss: 2.0018 - acc: 0.6188 - val_loss: 2.0655 - val_acc: 0.6156
Epoch 21/30
4477/4477 [==============================] - 8s 2ms/ste

...

Epoch 23/30
4477/4477 [==============================] - 7s 1ms/step - loss: 0.0339 - acc: 0.9913 - val_loss: 0.6657 - val_acc: 0.8817
Epoch 24/30
4477/4477 [==============================] - 7s 1ms/step - loss: 0.0354 - acc: 0.9909 - val_loss: 0.6604 - val_acc: 0.8864
Epoch 25/30
4477/4477 [==============================] - 7s 1ms/step - loss: 0.0362 - acc: 0.9904 - val_loss: 0.7321 - val_acc: 0.8779
Epoch 26/30
4477/4477 [==============================] - 7s 1ms/step - loss: 0.0386 - acc: 0.9895 - val_loss: 0.7023 - val_acc: 0.8798
Epoch 27/30
4477/4477 [==============================] - 7s 1ms/step - loss: 0.0259 - acc: 0.9938 - val_loss: 0.6601 - val_acc: 0.8902
Epoch 28/30
4477/4477 [==============================] - 7s 1ms/step - loss: 0.0228 - acc: 0.9948 - val_loss: 0.6398 - val_acc: 0.8942
Epoch 29/30
4477/4477 [==============================] - 7s 1ms/step - loss: 0.0272 - acc: 0.9937 - val_loss: 0.7085 - val_acc: 0.8876
Epoch 30/30
4477/4477 [==============================] 

4477/4477 [==============================] - 11s 2ms/step - loss: 0.0497 - acc: 0.9883 - val_loss: 0.6269 - val_acc: 0.8914
Epoch 22/30
4477/4477 [==============================] - 11s 2ms/step - loss: 0.0414 - acc: 0.9892 - val_loss: 0.5873 - val_acc: 0.9064
Epoch 23/30
4477/4477 [==============================] - 11s 2ms/step - loss: 0.0319 - acc: 0.9930 - val_loss: 0.5517 - val_acc: 0.9141
Epoch 24/30
4477/4477 [==============================] - 11s 2ms/step - loss: 0.0223 - acc: 0.9950 - val_loss: 0.5184 - val_acc: 0.9174
Epoch 25/30
4477/4477 [==============================] - 11s 2ms/step - loss: 0.0221 - acc: 0.9951 - val_loss: 0.5592 - val_acc: 0.9166
Epoch 26/30
4477/4477 [==============================] - 11s 2ms/step - loss: 0.0256 - acc: 0.9951 - val_loss: 0.5602 - val_acc: 0.9164
Epoch 27/30
4477/4477 [==============================] - 11s 2ms/step - loss: 0.0288 - acc: 0.9945 - val_loss: 0.6108 - val_acc: 0.9079
Epoch 28/30
4477/4477 [==============================] - 11s

Epoch 17/30
4477/4477 [==============================] - 6s 1ms/step - loss: 0.1845 - acc: 0.9420 - val_loss: 4.9028 - val_acc: 0.4960
Epoch 18/30
4477/4477 [==============================] - 6s 1ms/step - loss: 0.1694 - acc: 0.9469 - val_loss: 15.0603 - val_acc: 0.0050
Epoch 19/30
4477/4477 [==============================] - 6s 1ms/step - loss: 0.1423 - acc: 0.9549 - val_loss: 2.3819 - val_acc: 0.5938
Epoch 20/30
4477/4477 [==============================] - 6s 1ms/step - loss: 0.1428 - acc: 0.9548 - val_loss: 3.6072 - val_acc: 0.4952
Epoch 21/30
4477/4477 [==============================] - 6s 1ms/step - loss: 0.1352 - acc: 0.9571 - val_loss: 5.1318 - val_acc: 0.4621
Epoch 22/30
4477/4477 [==============================] - 6s 1ms/step - loss: 0.1430 - acc: 0.9542 - val_loss: 11.6281 - val_acc: 0.1588
Epoch 23/30
4477/4477 [==============================] - 6s 1ms/step - loss: 0.1192 - acc: 0.9623 - val_loss: 4.1881 - val_acc: 0.5953
Epoch 24/30
4477/4477 [==============================

4477/4477 [==============================] - 7s 2ms/step - loss: 0.0369 - acc: 0.9912 - val_loss: 0.5092 - val_acc: 0.9110s - loss: 0.0424 - acc: 0.989 - ETA: 
Epoch 15/30
4477/4477 [==============================] - 7s 2ms/step - loss: 0.0308 - acc: 0.9932 - val_loss: 0.5071 - val_acc: 0.9050s: 0.0289  - ETA: 0s - loss: 0.0306 - acc: 0.993
Epoch 16/30
4477/4477 [==============================] - 7s 2ms/step - loss: 0.0512 - acc: 0.9865 - val_loss: 0.5404 - val_acc: 0.9037 - loss: 0.0522 -
Epoch 17/30
4477/4477 [==============================] - 7s 2ms/step - loss: 0.0377 - acc: 0.9906 - val_loss: 0.5193 - val_acc: 0.9070 loss: 0.0395 - acc: 0. - ETA: 0s - loss: 0.0385 - 
Epoch 18/30
4477/4477 [==============================] - 7s 2ms/step - loss: 0.0240 - acc: 0.9943 - val_loss: 0.5232 - val_acc: 0.9090
Epoch 19/30
4477/4477 [==============================] - 7s 2ms/step - loss: 0.0279 - acc: 0.9932 - val_loss: 0.5162 - val_acc: 0.9106TA: 4s - loss: 0.0280 
Epoch 20/30
4477/4477 [====

4477/4477 [==============================] - 8s 2ms/step - loss: 0.1419 - acc: 0.9589 - val_loss: 0.5922 - val_acc: 0.8831
Epoch 9/30
4477/4477 [==============================] - 8s 2ms/step - loss: 0.0992 - acc: 0.9737 - val_loss: 0.5645 - val_acc: 0.8832
Epoch 10/30
4477/4477 [==============================] - 8s 2ms/step - loss: 0.0747 - acc: 0.9797 - val_loss: 0.5479 - val_acc: 0.8991
Epoch 11/30
4477/4477 [==============================] - 8s 2ms/step - loss: 0.0655 - acc: 0.9831 - val_loss: 0.5422 - val_acc: 0.8973
Epoch 12/30
4477/4477 [==============================] - 8s 2ms/step - loss: 0.0488 - acc: 0.9874 - val_loss: 0.5602 - val_acc: 0.9026
Epoch 13/30
4477/4477 [==============================] - 8s 2ms/step - loss: 0.0522 - acc: 0.9868 - val_loss: 0.5169 - val_acc: 0.9053
Epoch 14/30
4477/4477 [==============================] - 8s 2ms/step - loss: 0.0381 - acc: 0.9916 - val_loss: 0.5168 - val_acc: 0.90905s - loss: 0.0370 - acc: 0.9 - ETA: 5s - loss: 0.03
Epoch 15/30
4477/

In [23]:
print('The maximum validation accuracy reached is :- ',-1*search_result.fun)

The maximum validation accuracy reached is :-  0.9233529738480679


<div class="alert alert-block alert-info">
<b>Saving the model prepared:</b> We will now save the model fit above so that we can use it later
</div>

In [15]:
model = load_model(path_best_model)

In [21]:
from keras.models import load_model
model.save('chatbot_batman.h5')  

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 15, 100)           202800    
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 100)           400       
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1104)              2888064   
_________________________________________________________________
batch_normalization_2 (Batch (None, 1104)              4416      
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 12, 1104)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 12, 1104)          7321728   
_________________________________________________________________
batch_normalization_3 (Batch (None, 12, 1104)          4416      
__________

In [27]:
print('Learning rate   ',search_result.x[0])
print('Recurrent Nodes ',search_result.x[1])
print('Embedding Units ',search_result.x[2])

Learning rate    0.00035745406019704944
Recurrent Nodes  552
Embedding Units  100


<div class="alert alert-block alert-info">
<b>Chit chatting with my Bat-Bot developed :) :</b> We will try to perform some conversations with the bot developed.
</div>

In [17]:
def word_int(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [18]:
def predict_sequence(model, tokenizer, value):
    prediction = model.predict(value, verbose=0)[0]
    integers = [np.argmax(vector) for vector in prediction]
    target = []
    for i in integers:
        word = word_int(i, tokenizer)
        if word is None:
            break
        target.append(word)
    return ' '.join(target)

In [19]:
while True:
    ques_input = input('Bat-Query :- Please enter your Query!  ')
    if ques_input == 'Thanks! Bye!':
        break
    question = [ques_input]
    encode_ques = encode_sequences(ques_tokenizer, question_length, question)
    translation = predict_sequence(model, ans_tokenizer, encode_ques)
    print('Bat-Bot:-' ,translation)

Bat-Query :- Please enter your Query!  Can I talk to your manager?
Bat-Bot:- im at your service
Bat-Query :- Please enter your Query!  What's your name ?
Bat-Bot:- i dont have family name
Bat-Query :- Please enter your Query!  You are so confusing me dear!
Bat-Bot:- i think i might have gotten lost there
Bat-Query :- Please enter your Query!  ya you have ..Certainly 
Bat-Bot:- excellent
Bat-Query :- Please enter your Query!  are you being serious with me .. i wanna talk to your manager
Bat-Bot:- im at your service
Bat-Query :- Please enter your Query!  You know I have programmed you and it took me one complete day for that !
Bat-Bot:- lets keep
Bat-Query :- Please enter your Query!  keep what ?
Bat-Bot:- i think have might have gotten
Bat-Query :- Please enter your Query!  ya you got me !
Bat-Bot:- i there
Bat-Query :- Please enter your Query!  you are my favorite !
Bat-Bot:- i dont need you you
Bat-Query :- Please enter your Query!  why?
Bat-Bot:- sorry about at
Bat-Query :- Please en